In [ ]:
# Cell 0

network = "mainnet"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"sejong": {"iconservice": "https://sejong.net.solidwallet.io", "nid": 83},
"hannam": {"iconservice": "https://hannam.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.144.108.38:9000",        "nid": 3}}

env = connections[network]

In [ ]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from repeater import retry
import json


In [ ]:
# Cell 2

icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

In [ ]:
NONCE = 100
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

# checking transaction result
def check_tx_result(res):
    if res['status'] == 0:
        print("Tx Failed, Message: " + res['failure']['message'])
    else:
        print("Success")
        
        
# GOVERNANCE_SCORE = "cx06eaddea58321a98bd0e1c9bd70f4680d4f3a73a"
# DEX_SCORE = "cx113466143604608cbf58d84674d9360b29f08530"
# LOANS_SCORE = "cx2148bdb92054b2306076f3579c0ab7fced91a9ca"


# MAINNET ADDRESS
GOVERNANCE_SCORE = "cx44250a12074799e26fdeee75648ae47e2cc84219"
DEX_SCORE = "cx481f7e10c7c4fc59d6ac68472018f10977a91b39"
LOANS_SCORE = "cx66d4d90f5f113eba575bf793570135f9b10cece1"



In [ ]:
with open("./keystores/staking_test.pwd", "r") as f:
    key_data = f.read()
deployer_wallet = KeyWallet.load("./keystores/staking_test.json", key_data)

#using private key to load test wallet to perform transactions in case of need
# tester_wallet = KeyWallet.load(bytes.fromhex(""))
# tester_wallet.get_address()

In [ ]:
def deploy_contract(wallet, _to_address, _name, params):
    deploy_transaction = DeployTransactionBuilder() \
        .from_(wallet) \
        .to(_to_address) \
        .nid(NID) \
        .params(params) \
        .version(3) \
        .nonce(100) \
        .content_type("application/zip") \
        .content(gen_deploy_data_content(_name)) \
        .build()

    estimate_step = icon_service.estimate_step(deploy_transaction)
    step_limit = estimate_step + 40000000
    signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)

    return icon_service.send_transaction(signed_transaction)


def send_tx(wallet, _to_address, method, params, value):
    transaction = CallTransactionBuilder() \
        .from_(wallet.get_address()) \
        .to(_to_address) \
        .value(value) \
        .step_limit(10000000) \
        .nid(NID) \
        .nonce(100) \
        .method(method) \
        .params(params) \
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)

    return get_tx_result(tx_hash)


def call_tx(_to_address, method, params):
    call = CallBuilder() \
        .to(_to_address) \
        .method(method) \
        .params(params) \
        .build()

    return icon_service.call(call)

#swap function for DEX
def swap(from_token, to_token, amount,wallet):
    data2 = json.dumps({"method": "_swap", "params": {"toToken":str(to_token)}})
    params = {'_to': contract_addresses['dex'], '_value': amount, '_data': data2.encode()}
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(from_token)\
        .step_limit(10000000)\
        .value(0)\
        .nid(NID)\
        .nonce(100)\
        .method("transfer")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    return get_tx_result(tx_hash)

In [ ]:
# Getting all the SCORE address from GOVERNANCE SCORE
# The result shouldn't have router and feehandler in keys
contract_addresses = call_tx(GOVERNANCE_SCORE, 'getAddresses', {})
print(contract_addresses)

In [ ]:
# Taking loans and swapping tokens on dex 

print("Taking loans")
res = send_tx(deployer_wallet, contract_addresses['loans'], "depositAndBorrow", {'_asset': 'bnUSD', '_amount': 11 * 10 **18}, 50 * 10 **18)
check_tx_result(res)


res = swap(contract_addresses['bnUSD'], contract_addresses['sicx'],5*10**18, deployer_wallet)
check_tx_result(res)
#print txHash incase of need
# print(res['txHash'])

# res = swap(contract_addresses['sicx'], contract_addresses['bnUSD'],15*10**18, deployer_wallet)
# check_tx_result(res)

# res = swap(data['bnUSD'], usds_score,50*10**18, deployer_wallet)
# check_tx_result(res)

# res = swap(data['bnUSD'], data['sicx'],50*10**18, deployer_wallet)
# check_tx_result(res)

# res = swap(data['sicx'], omm_score,10*10**18, deployer_wallet)
# check_tx_result(res)

# res = swap(omm_score, data['sicx'],5*10**18, deployer_wallet)
# check_tx_result(res)


# PHASE 1

In Phase 1 , 
We deploy:
    Router contract,
    Fee handler contract and
    Governance Contract (update)

## DEPLOY ROUTER CONTRACT

In [ ]:
# We need to set all the staking sicx and dex address on router on_install 
res = get_tx_result(deploy_contract(deployer_wallet.get_address(), GOVERNANCE_ADDRESS, './core_contracts/router',
                                   {"_governance": GOVERNANCE_SCORE}))
print('scoreAddress:', res.get('scoreAddress', ''))
router_score = res.get('scoreAddress', '')
check_tx_result(res)

## UPDATE GOVERNANCE CONTRACT

In [ ]:
tx = get_tx_result(
    deploy_contract(deployer_wallet.get_address(), GOVERNANCE_SCORE, './core_contracts/governance', {}))
print('scoreAddress:', tx.get('scoreAddress', ''))
check_tx_result(tx)

# In this readonly-call , the feehandler and router score should show None.
print(call_tx(GOVERNANCE_SCORE, 'getAddresses', {}))


## DEPLOY FEE HANDLER

In [ ]:
res = get_tx_result(deploy_contract(deployer_wallet.get_address(), GOVERNANCE_ADDRESS, './core_contracts/fees',
                                   {"_governance": GOVERNANCE_SCORE}))
print('scoreAddress:', res.get('scoreAddress', ''))
FEEHANDLER_SCORE = res.get('scoreAddress', '')
check_tx_result(res)

# PHASE 2 (Once the contract is accepted)

We set the fee handler and router addresses in governance contract and also sets all the function of fee handler.

#### SET ROUTER ADDRESS AND FEEHANDLER ADDRESS IN GOVERNANCE

In [ ]:
contract_addresses['router'] = router_score
contract_addresses['feehandler'] = FEEHANDLER_SCORE
res = send_tx(deployer_wallet, GOVERNANCE_SCORE, "setAddresses", {"_addresses": contract_addresses}, 0)
check_tx_result(res)

# the read-only fn should return the fee handler and router address instead of None.
print(call_tx(GOVERNANCE_SCORE, 'getAddresses', {}))

#### SET ACCEPTED DIVIDENDS TOKEN

In [ ]:
# call the tx through Governance SCORE.

res2 = send_tx(deployer_wallet, GOVERNANCE_SCORE, "setAcceptedDividendTokens", {'_tokens': [contract_addresses['sicx'],
            contract_addresses['bnUSD'],
            contract_addresses["baln"]]}, 0)
check_tx_result(res2)
call_tx(FEEHANDLER_SCORE, 'getAcceptedDividendTokens', {})

#### SET FEE PROCESSING INTERVAL

In [ ]:
res = send_tx(deployer_wallet, GOVERNANCE_SCORE, "setFeeProcessingInterval", {"_interval": 100}, 0)
check_tx_result(res)
call_tx(FEEHANDLER_SCORE, 'getFeeProcessingInterval', {})

#### SET ROUTE

In [ ]:
toToken = contract_addresses['baln']     
#sejong_address
# omm_score = "cx683c78174c740c3c6215de0c3ead1fad024324d6"
# usds_score ="cx2e008873de78f5bf82008a46b529d94924d9b6ae" 
# iusdc_score = "cxb2c075e9130440dd64e6bfd3fe09e5c629f6e183"

#mainnet address
usdc_score = "cxbb2871f468a3008f80b08fdde5b8b951583acf06"
iusdc_score = "cxae3034235540b924dfcc1b45836c293dcc82bfb7"
omm_score = "cx1a29259a59f463a67bb2ef84398b30ca56b5830a"
craft_score = "cx2e6d0fc0eca04965d06038c8406093337f085fcf"
metanyx_score = "cx369a5f4ce4f4648dfc96ba0c8229be0693b4eca2"
iusdt_score = "cx3a36ea1f6b9aa3d2dd9cb68e8987bcc3aabaaa88"
gbet_score = "cx6139a27c15f1653471ffba0b4b88dc15de7e3267"

# FOR OMM
# fromToken = omm_score
# path = [ contract_addresses['sicx'],
#         contract_addresses['baln']
#         ]
# res = send_tx(deployer_wallet, GOVERNANCE_SCORE, "setRoute",  {'_fromToken':fromToken, '_toToken':toToken, '_path': path}, 0)
# check_tx_result(res)
# print(call_tx(FEEHANDLER_SCORE, 'getRoute', {"_fromToken":fromToken,"_toToken":contract_addresses['baln']}))

# FOR usds
# fromToken = usds_score
# path = [ contract_addresses['bnUSD'],
#         contract_addresses['baln']
#         ]
# res = send_tx(deployer_wallet, GOVERNANCE_SCORE, "setRoute",  {'_fromToken':fromToken, '_toToken':toToken, '_path': path}, 0)
# check_tx_result(res)
# print(call_tx(FEEHANDLER_SCORE, 'getRoute', {"_fromToken":fromToken,"_toToken":contract_addresses['baln']}))

# FOR iusdc
# fromToken = iusdc_score
# path = [ contract_addresses['bnUSD'],
#         contract_addresses['baln']
#         ]
# res = send_tx(deployer_wallet, GOVERNANCE_SCORE, "setRoute",  {'_fromToken':fromToken, '_toToken':toToken, '_path': path}, 0)
# check_tx_result(res)
# print(call_tx(FEEHANDLER_SCORE, 'getRoute', {"_fromToken":fromToken,"_toToken":contract_addresses['baln']}))

# FOR craft
# fromToken = craft_score
# path = [ data['sicx'],
#         data['baln']
#         ]
# send_tx(deployer_wallet, FEEHANDLER_SCORE, "setRoute",  {'_fromToken':fromToken, '_toToken':toToken, '_path': path}, 0)

# FOR metanyx
# fromToken = metanyx_score
# path = [ data['bnUSD'],
#         data['baln']
#         ]
# send_tx(deployer_wallet, FEEHANDLER_SCORE, "setRoute",  {'_fromToken':fromToken, '_toToken':toToken, '_path': path}, 0)

# FOR iusdt
# fromToken = iusdt_score
# path = [ data['bnUSD'],
#         data['baln']
#         ]
# send_tx(deployer_wallet, FEEHANDLER_SCORE, "setRoute",  {'_fromToken':fromToken, '_toToken':toToken, '_path': path}, 0)

# FOR gbet
fromToken = gbet_score
path = [ contract_addresses['bnUSD'],
        contract_addresses['baln']
        ]
send_tx(btest_wallet, GOVERNANCE_SCORE, "setRoute",  {'_fromToken': fromToken, '_toToken': toToken, '_path': path}, 0)


In [ ]:
send_tx(btest_wallet, contract_addresses['feehandler'], "add_allowed_address",  {'address': gbet_score}, 0)


## UPDATE DEX CONTRACT

In [ ]:
# We need to set fee handler address on on_update.
tx = get_tx_result(
    deploy_contract(deployer_wallet.get_address(), DEX_SCORE, './core_contracts/dex', {}))
print('scoreAddress:', tx.get('scoreAddress', ''))
check_tx_result(tx)

## UPDATE LOANS CONTRACT

In [ ]:
tx = get_tx_result(
    deploy_contract(deployer_wallet.get_address(), LOANS_SCORE, './core_contracts/loans', {}))
print('scoreAddress:', tx.get('scoreAddress', ''))
check_tx_result(tx)


In [ ]:
print("Taking loans")
res = send_tx(deployer_wallet, contract_addresses['loans'], "depositAndBorrow", {'_asset': 'bnUSD', '_amount': 100 * 10 **18}, 500 * 10 **18)
check_tx_result(res)

#Swapping bnUSD to sICX
res = swap(contract_addresses['bnUSD'], contract_addresses['sicx'],5*10**18,deployer_wallet)
check_tx_result(res)